In [0]:
import numpy as np
import pandas as pd
from sklearn import metrics
from tqdm import tqdm
import spacy
# import en_core_web_lg

import nltk
# nltk.download('wordnet')

from collections import Counter, defaultdict

from nltk.tokenize import word_tokenize
from nltk import pos_tag

from collections import defaultdict
from nltk.corpus import wordnet as wn
from tqdm import tqdm

nlp = spacy.load('en')

from spacy import displacy
import en_core_web_sm

import re

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# nlp = spacy.load('en_core_web_lg', parse=True, tag=True, entity=True)

In [0]:
#load the dataset

trainDf = pd.read_csv("./Dataset/train.csv",encoding = "utf-8")

In [5]:
trainDf.head()

,ID,reviewText,overall
0,1,These are so wonderful to have in the car in t...,5
1,2,XL fits perfect on me over armored riding jack...,5
2,3,Since I had just noticed my wiper blades neede...,3
3,4,I am very satisfied with G110v2. It is comfor...,5
4,5,This amp did a good job for its rating. It was...,3


In [6]:
trainDf.isna().sum()

ID             0
reviewText    26
overall        0
dtype: int64

In [0]:
cleanup_re = re.compile('[^a-zA-Z]+')

def cleanup(sentence):
    sentence = str(sentence)
    sentence = re.sub(r'\s+',' ',sentence) #\s is for white spaces
    sentence = re.sub('[\d]','',sentence)  #\d is for digits
    #sentence = sentence.lower()
    sentence = cleanup_re.sub(' ', sentence).strip()
    #sentence = " ".join(nltk.word_tokenize(sentence))
    return sentence

trainDf["text"] = trainDf["reviewText"].apply(cleanup)

In [0]:
trainDf['text'].dropna(inplace=True)

In [9]:
trainDf.shape

(77075, 4)

In [0]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"didnt": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldve": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"youre": "you are",
"you've": "you have",
"inroom": "in room"

}

In [0]:
def expand_contractions(text):
    for word in text.split():
        if word in contractions:
            text = text.replace(word, contractions[word])
    return text

In [12]:
trainDf['text'] = [expand_contractions(re.sub('’', "'", text)) for text in trainDf['text']]
trainDf['text'][166]

'I was surprised and delighted with this excellent Norwegian riff on the found footage genre This time we have all the clips from a student s documentary on trolls put in chronologically order Shot in verit eacute style in the vein of films like CLOVERFIELD TROLLHUNTER follows three persistent Norwegian film students as they travel to a remote mountain region in hopes of capturing a real life troll on camera a creature whose existence has been understandably covered up for decades by the local government They claim there s nothing to worry about it s just a problem with bears making trouble in the more remote mountains and forests of Norway Yes there have been attacks Local hunters and the film students don t believe the disinformation and their determination pays off when they stalk a shadowy poacher and finally get him to agree to let them record his hunt their encounters wit the fearsome giants and the troll hunter s heroics This is the coolest monster movie I ve seen in a long time

In [0]:
# Define function to handle accented characters
import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #https://docs.python.org/2/library/unicodedata.html
    return text

In [0]:
trainDf['text'] = [remove_accented_chars(text) for text in trainDf['text']]

In [0]:
from sklearn.model_selection import train_test_split
#split the dataset intro training an dvalidation datasets

X_train, X_test, y_train, y_test = train_test_split(trainDf['text'], trainDf['overall'], test_size = 0.30)

In [16]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(53952,)
(53952,)
(23123,)
(23123,)


In [17]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [18]:
# Statement that we used on the top
sample_statement = ["Jack while talking over the cell phone entered the prison cell to extract blood cell samples of Jill and made an entry in the excel cell about the blood sample collection."]
# Extract ELMo features 
embeddings = elmo(sample_statement, signature="default", as_dict=True)["elmo"]
embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([Dimension(1), Dimension(31), Dimension(1024)])

In [19]:
from tensorflow.keras.layers import Input, Lambda, Bidirectional, Dense, Dropout,LSTM
from tensorflow.keras.models import Model
from keras import backend as K
def ELMoEmbedding(input_text):
    return elmo(tf.reshape(tf.cast(input_text, tf.string), [-1]), signature="default", as_dict=True)["elmo"]
def build_model():
    input_layer = Input(shape=(1,), dtype="string", name="Input_layer")
    embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ), name="Elmo_Embedding")(input_layer)
    BiLSTM = Bidirectional(LSTM(2, return_sequences= False, recurrent_dropout=0.2, dropout=0.2), name="BiLSTM")(embedding_layer)
    Dense_layer_1 = Dense(16, activation='relu')(BiLSTM)
    Dropout_layer_1 = Dropout(0.5)(Dense_layer_1)
    Dense_layer_2 = Dense(8, activation='relu')(Dropout_layer_1)
    Dropout_layer_2 = Dropout(0.5)(Dense_layer_2)
    output_layer = Dense(1, activation='sigmoid')(Dropout_layer_2)
    model = Model(inputs=[input_layer], outputs=output_layer, name="BiLSTM with ELMO Embeddings")
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model
elmo_BiDirectional_model = build_model()

Using TensorFlow backend.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "BiLSTM with ELMO Embeddings"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
Elmo_Embedding (Lambda)      (None, None, 1024)        0         
_________________________________________________________________
BiLSTM (Bidirectional)       (None, 4)                 16432     
_________________________________________________________________
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_1 (Dropout)          (None, 8) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
with tf.Session() as session:
 K.set_session(session)
 session.run(tf.global_variables_initializer()) 
 session.run(tf.tables_initializer())
 model_elmo = elmo_BiDirectional_model.fit(X_train, y_train, epochs=1, batch_size=1024)
 train_prediction = elmo_BiDirectional_model.predict(X_train)
 #model_elmo.save_weights('./model_elmo_neural_network_weights.h5')


Train on 53952 samples


In [0]:
# Accuracy
train_prediction = train_prediction.argmax(axis=-1)

metrics.accuracy_score(train_prediction, y_train)